In [ ]:
import torch
import numpy as np
from rfm import LaplaceRFM  # Đảm bảo rằng LaplaceRFM được import đúng cách

torch.set_default_dtype(torch.float32)
torch.manual_seed(0)

# Define target function
def fstar(X):
    return torch.cat([ 
        (X[:, 0] > 0)[:, None],  # Sử dụng broadcasting để tạo nhãn từ cột đầu tiên
        (X[:, 1] < 0.1)[:, None]  # Tạo nhãn từ cột thứ hai
    ], axis=1).type(X.type())  # Kết hợp nhãn thành ma trận 2 cột

# Create low rank data
n = 4000
d = 100
torch.manual_seed(0)

# Tạo dữ liệu huấn luyện và kiểm tra dưới dạng tensor PyTorch
X_train = torch.randn(n, d)
X_test = torch.randn(n, d)

print(f"Shape of X_train: {X_train.shape}")

# Tạo nhãn cho dữ liệu
y_train = fstar(X_train)
y_test = fstar(X_test)

# Kiểm tra và chuyển sang GPU nếu có sẵn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Di chuyển dữ liệu về đúng device (GPU hoặc CPU)
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

# Khởi tạo mô hình LaplaceRFM
model = LaplaceRFM(bandwidth=1., diag=False, centering=False)

# Di chuyển mô hình lên GPU nếu có
model = model.to(device)
model.max_lstsq_size = 2000
# Huấn luyện mô hình với NMF method
model.fit(
    (X_train, y_train),
    (X_test, y_test),
    method='nmf',  # Chọn phương pháp NMF
    epochs=15,
    iters=5,
    classification=True,  # Phân loại
    verbose=True,
    prefit_nmf=True
)


c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Shape of X_train: torch.Size([4000, 100])
Using device: cuda
Time taken to prefit nmf with 70000 points: 0.2130889892578125 seconds
(0 epochs, 0.0s)	train mse: 0.0000 	val mse: 0.5129
	train acc: 0.5630 	val acc: 0.4397
New best multiclass-acc: 0.4397
Round 0, Test Acc: 43.97%
Round 0, Test MSE: 0.5129
Optimal M batch size:  669
Using batch size of 669
Sampling AGOP on maximum of 20070 total points


100%|██████████| 6/6 [00:00<00:00, 153.83it/s]

Time taken to prefit nmf with 70000 points: 0.09864068031311035 seconds


(0 epochs, 0.0s)	train mse: 0.0000 	val mse: 0.5001
	train acc: 0.5620 	val acc: 0.4770
New best multiclass-acc: 0.4770
Round 1, Test Acc: 47.70%
Round 1, Test MSE: 0.5001
Optimal M batch size:  669
Using batch size of 669
Sampling AGOP on maximum of 20070 total points


...

Time taken to prefit nmf with 70000 points: 0.10752606391906738 seconds


(0 epochs, 0.0s)	train mse: 0.0000 	val mse: 0.0780
	train acc: 0.7830 	val acc: 0.7345
New best multiclass-acc: 0.7345
Round 4, Test Acc: 73.45%
Round 4, Test MSE: 0.0780
Optimal M batch size:  669
Using batch size of 669
Sampling AGOP on maximum of 20070 total points


100%|██████████| 6/6 [00:00<00:00, 113.19it/s]


Time taken to prefit nmf with 70000 points: 0.13137269020080566 seconds
(0 epochs, 0.0s)	train mse: 0.0000 	val mse: 0.0642
	train acc: 0.5350 	val acc: 0.7375
New best multiclass-acc: 0.7375
Final MSE: 0.0642
Final Test Acc: 73.75%


tensor(0.0642, device='cuda:0')